In [ ]:
# Hugging Face 라이브러리 적용 - 기계 번역 모델
# AI HUB 방송 다국어 번역 데이터셋

In [7]:
import torch
import numpy as np
import glob, json, re, os, random, csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__, device)

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
print("빌드 정보:", torch.__version__)
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

2.8.0+cu129 cuda
CUDA 사용 가능 여부: True
PyTorch CUDA 버전: 12.9
빌드 정보: 2.8.0+cu129
사용 중인 GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [8]:
# 데이터셋 전처리
ko_lines, en_lines = [], []
folders = [ # 폴더 리스트 정의
    './llm_data/aihub_broadcast_translation/aihub_documentary_esko_translation/*.json',
    './llm_data/aihub_broadcast_translation/aihub_etc_esko_translation/*.json',
    './llm_data/aihub_broadcast_translation/aihub_movie_esko_translation/*.json'
]

# 모든 JSON 읽기
for folder in folders:
    for path in glob.glob(folder): # 특정 디렉토리에서 지정한 패턴과 일치하는 모든 파일 경로를 리스트로 반환
        with open(path, encoding='utf-8') as f:
            try:
                data = json.load(f) # 파일 전체 로드(dict 구조)
            except json.JSONDecodeError:
                continue

            # 원문(영어), 최종번역문(한국어) 추출
            en = data.get('원문')
            ko = data.get('최종번역문')

            if en and ko and ko != 'N/A':
                en_lines.append(en.strip())
                ko_lines.append(ko.strip())

# 1. Detokenize 함수 정의
def detokenize_sentence(sentence: str) -> str:
    sentence = sentence.strip()
    sentence = re.sub(r"\s+([?.!,])", r"\1", sentence)  # " ?" → "?"
    sentence = re.sub(r"\s+", " ", sentence)            # 여러 공백 → 하나
    return sentence

# 2. 데이터셋 전처리
en_lines = [detokenize_sentence(s) for s in en_lines]
ko_lines = [detokenize_sentence(s) for s in ko_lines]


print(f'총 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')

총 문장쌍 개수: 121124, 121124


In [9]:
# 데이터 전처리 - 중복 제거 및 순서 유지
# pairs = list(set(zip(en_lines, ko_lines)))
# en_lines, ko_lines = zip(*pairs) # 다시 분리
seen = set()
pairs = []
for en, ko in zip(en_lines, ko_lines):
    if (en, ko) not in seen:
        # 새로운 문장쌍을 집합에 기록, 이후 같은 문장쌍이 나오면 if 조건에서 걸러져 추가되지 않는다
        seen.add( (en, ko) )
        
        # 중복이 아닌 문장쌍을 리스트에 추가, 원래 순서대로 중복 없는 문장쌍 리스트가 만들어 진다
        # - pairs는 [("Hello","안녕"), ("Goodbye","잘가")] 
        pairs.append( (en, ko) )

# 이를 다시 분리 - 영어 문장들만 모아 ("Hello","Goodbye"), 한국어 문장들만 모아 ("안녕","잘가")
en_lines, ko_lines = zip(*pairs)

print(f'중복 제거 후 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')

중복 제거 후 문장쌍 개수: 121115, 121115


In [10]:
# 데이터 전처리 - 샘플링 추가

# 샘플링 최대 50,000 문장만 사용
# sample_size = 50000
sample_size = 100000
if len(ko_lines) > sample_size:
    indices = random.sample(range(len(ko_lines)), sample_size)
    ko_lines = [ ko_lines[i] for i in indices ]
    en_lines = [ en_lines[i] for i in indices ]

print(f'샘플링 후 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')

샘플링 후 문장쌍 개수: 100000, 100000


In [11]:
# 데이터 전처리 - 저장
out_dir = './llm_data/aihub_broadcast_translation'

# 폴더 없을시 생성
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)
    print(f'폴더 생성 완료: {out_dir}')
else:
    print(f'이미 존재하는 폴더: {out_dir}')

ko_path = f'{out_dir}/train_ko.txt'
en_path = f'{out_dir}/train_en.txt'

with open(ko_path, 'w', encoding='utf-8') as fko, \
    open(en_path, 'w', encoding='utf-8') as fen:
    for k, e in zip(ko_lines, en_lines):
        fko.write(k + '\n')
        fen.write(e + '\n')
print('저장 완료', ko_path, en_path)

이미 존재하는 폴더: ./llm_data/aihub_broadcast_translation
저장 완료 ./llm_data/aihub_broadcast_translation/train_ko.txt ./llm_data/aihub_broadcast_translation/train_en.txt


In [53]:
# AI Hub 방송 데이터셋(train_ko.txt, train_en.txt) -> CSV로 변환해 파인 튜닝
# 원본 데이터는 train_ko.txt, train_en.txt로 분리 -> 병렬 문장쌍을 만들어야 함
# 파인튜닝시 양방향 번역을 지원하려면 같은 문장쌍은 en->ko, ko->en 두방향으로 모두 포함해야 함
# CSV 구조 예시
# src,tgt,src_lang,tgt_lang
# You can buy it from a convenience store try it out.,편의점에서 사실 수 있으니 시도해보시길 바랍니다.,en,ko
# 편의점에서 사실 수 있으니 시도해보시길 바랍니다.,You can buy it from a convenience store try it out.,ko,en
# She frees him and takes him as her navigator.,그녀는 그를 풀어주고 조종자로써 그를 데려간다.,en,ko
# 그녀는 그를 풀어주고 조종자로써 그를 데려간다.,She frees him and takes him as her navigator.,ko,en
# Iyengar's belief in Gandhi's philosophy is so deep that he can't even open his laptop without remorse.,간디의 철학에 대한 아이옌가르의 믿음은 너무 깊어서, 그는 심지어 그의 노트북도 양심의 가책 없이 열 수 없다.,en,ko
# 간디의 철학에 대한 아이옌가르의 믿음은 너무 깊어서, 그는 심지어 그의 노트북도 양심의 가책 없이 열 수 없다.,Iyengar's belief in Gandhi's philosophy is so deep that he can't even open his laptop without remorse.,ko,en
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# 전체 데이터 로드 - AI Hub 방송 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/aihub_broadcast_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/aihub_broadcast_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines = f_en.read().splitlines()
    ko_lines = f_ko.read().splitlines()

# 데이터 개수 제한 (예: 100개)
limit = 1000
en_lines = en_lines[:limit]
ko_lines = ko_lines[:limit]

# train/valid split(90 : 10)
split_idx = int(len(en_lines) * 0.9)
train_en, valid_en = en_lines[:split_idx], en_lines[split_idx:]
train_ko, valid_ko = ko_lines[:split_idx], ko_lines[split_idx:]

print(len(train_en))
print(len(valid_en))

900
100


In [54]:
# 병렬 데이터 생성

# train.csv 생성
with open('./llm_data/aihub_broadcast_translation/train.csv', 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang'])

    for en, ko in zip(train_en, train_ko):
        en, ko = en.strip(), ko.strip()
        if not en or not ko:
            continue

        # 영어 -> 한국어
        writer.writerow([en, ko, 'en', 'ko'])
        # 한국어 -> 영어
        writer.writerow([ko, en, 'ko', 'en'])

# valid.csv 생성
with open('./llm_data/aihub_broadcast_translation/valid.csv', 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang'])

    for en, ko in zip(valid_en, valid_ko):
        en, ko = en.strip(), ko.strip()
        if not en or not ko:
            continue
        
        # 영어 -> 한국어
        writer.writerow([en, ko, 'en', 'ko'])
        # 한국어 -> 영어
        writer.writerow([ko, en, 'ko', 'en'])

In [55]:
# 토크나이저 전처리
from datasets import load_dataset
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# 1. tokenizer 로드
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 2. 전처리 함수 (동적 tgt_lang 설정)
def preprocess_function(examples):
    inputs = examples["src"]
    targets = examples["tgt"]
    tgt_langs = examples["tgt_lang"]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    labels_list = []
    for text, lang in zip(targets, tgt_langs):
        if lang in tokenizer.lang_code_to_id:
            tokenizer.tgt_lang = lang
        else:
            tokenizer.tgt_lang = "en"  # 기본값

        with tokenizer.as_target_tokenizer():
            labels = tokenizer(text, max_length=128, truncation=True, padding="max_length")
        labels_list.append(labels["input_ids"])

    model_inputs["labels"] = labels_list
    return model_inputs

In [56]:
# 데이터셋 불러오기
dataset = load_dataset(
    "csv",
    data_files={
        "train": "./llm_data/aihub_broadcast_translation/train.csv",
        "validation": "./llm_data/aihub_broadcast_translation/valid.csv"
    }
)

# 토크나이즈 적용
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [57]:
# 모델 로드
base_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

# LoRA 설정
lora_config = LoraConfig(
    r=8,              # 랭크 크기
    lora_alpha=32,    # 스케일링 계수
    lora_dropout=0.1, # 드롭아웃
    target_modules=["q_proj", "v_proj"]  # Attention 모듈에 적용
)

# LoRA 모델 생성
model = get_peft_model(base_model, lora_config)

In [58]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./results_lora",
    eval_strategy="epoch", # 구버전
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

In [59]:
# 학습 실행
trainer.train()

Epoch,Training Loss,Validation Loss
1,9.022100,6.771487
2,6.731700,6.245128
3,6.424600,6.196653


TrainOutput(global_step=339, training_loss=7.278763582572824, metrics={'train_runtime': 5912.2558, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.057, 'total_flos': 1467687842611200.0, 'train_loss': 7.278763582572824, 'epoch': 3.0})

In [60]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 영어 → 한국어
text = "It is literally the basis of life."
inputs = tokenizer(text, return_tensors="pt").to(device)   # 입력도 GPU로 이동
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)
print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))

# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
inputs = tokenizer(text, return_tensors="pt").to(device)   # 입력도 GPU로 이동
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)
print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))

EN→KO: 그것은 문자 그대로 삶의 기초입니다.
KO→EN: It is literally the basis of life.


In [61]:
# LoRA 적용된 모델 저장
model.save_pretrained("./llm_models/lora_translation_model")
tokenizer.save_pretrained("./llm_models/lora_translation_model")

('./llm_models/lora_translation_model\\tokenizer_config.json',
 './llm_models/lora_translation_model\\special_tokens_map.json',
 'llm_models\\lora_translation_model\\vocab.json',
 'llm_models\\lora_translation_model\\sentencepiece.bpe.model',
 './llm_models/lora_translation_model\\added_tokens.json')

In [62]:
from peft import PeftModel

# 원본 M2M100 모델 로드
base_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

# LoRA 어댑터 붙여서 불러오기
model = PeftModel.from_pretrained(base_model, "./llm_models/lora_translation_model")

# 토크나이저도 불러오기
tokenizer = M2M100Tokenizer.from_pretrained("./llm_models/lora_translation_model")

# 디바이스 맞추기
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [71]:
# 영어 → 한국어
text = "It is literally the basis of life."
texts = [
    "The weather is very good today.",
    "I am studying machine learning.",
    "Artificial intelligence is changing the world.",
    "Deep learning is a subset of machine learning.",
    "Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day.",
    "When women replaced men, they earned much less money than them, which was a real problem.",
    "It is literally the basis of life.",
    "It's hard to believe, but the People's Artist of Russia, singer and TV presenter Nadezhda Babkina is 70 years old!",
    "Arriving in the USA, Nuno is assigned the teacher Jane Dayle Haddon as a &quot;tutor&quot;.",
    "This mediated dialogue allows Stefania and Andrea to reflect on their relationship and, when the game is revealed, to resume it with greater awareness."

]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("EN→KO:")
for i, output in enumerate(outputs):
    print(f"{texts[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
texts_ko = [
    "오늘 날씨는 매우 좋습니다.",
    "저는 머신러닝을 공부하고 있습니다.",
    "인공 지능은 세계를 변화시킵니다.",
    "딥러닝은 머신 러닝의 하위 세트입니다.",
    "지엠 판매원인 로베르토는 그가 매일 편지를 쓰는 오르넬라 무티를 열렬히 사랑한다.",
    "여성이 남성을 대체했을 때, 그들은 남성보다 훨씬 적은 돈을 벌게 되었고 이것이 진짜 문제였습니다.",
    "말 그대로 삶의 기초입니다.",
    "믿기 어렵겠지만, 러시아의 인민 예술가이자 가수이자 텔레비전 진행자인 나데즈다 밥키나는 70세입니다!",
    "미국에 도착한 누노는 제인 데일 해든 선생님으로 임명되었다.",
    "이 중재된 대화를 통해 스테파니아와 안드레아는 그들의 관계에 대해 반성하고 게임이 공개되면 더 큰 인식으로 게임을 재개할 수 있습니다."

]
inputs = tokenizer(texts_ko, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("\nKO→EN:")
for i, output in enumerate(outputs):
    print(f"{texts_ko[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


EN→KO:
The weather is very good today. → 오늘날 날씨는 매우 좋습니다.
I am studying machine learning. → 저는 기계 학습을 공부하고 있습니다.
Artificial intelligence is changing the world. → 인공 지능이 세상을 바꾸고 있다.
Deep learning is a subset of machine learning. → 깊은 학습은 기계 학습의 하위 세트입니다.
Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day. → GM 판매자 인 로베로 (Roberto)는 매일 글을 쓰는 오넬라 무티 (Ornella Muti)와 미친 사랑에 빠져 있습니다.
When women replaced men, they earned much less money than them, which was a real problem. → 여성들이 남자를 대체했을 때, 그들은 그들보다 훨씬 적은 돈을 벌었습니다.
It is literally the basis of life. → 그것은 문자 그대로 삶의 기초입니다.
It's hard to believe, but the People's Artist of Russia, singer and TV presenter Nadezhda Babkina is 70 years old! → 믿기 어려운 일이지만, 러시아의 민간 아티스트, 노래가자 TV 선물자 나데지다 바비키나가 70세입니다!
Arriving in the USA, Nuno is assigned the teacher Jane Dayle Haddon as a &quot;tutor&quot;. → 미국에 도착했을 때, Nuno는 교사 Jane Dayle Haddon에게 &quot;교사&quot;로 임명됩니다.
This mediated dialogue allows Stefania and Andrea to refle

In [74]:
import sacrebleu

# 영어 → 한국어 평가
references_ko = [
    "오늘 날씨는 매우 좋습니다.",
    "저는 머신러닝을 공부하고 있습니다.",
    "인공지능은 세상을 변화시키고 있습니다.",
    "딥러닝은 머신러닝의 하위 집합입니다.",
    "지엠 판매원 로베르토는 오르넬라 무티를 열렬히 사랑하며 매일 편지를 씁니다."
]

hypotheses_ko = [
    "오늘날 날씨는 매우 좋습니다.",
    "저는 기계 학습을 공부하고 있습니다.",
    "인공 지능이 세상을 바꾸고 있다.",
    "깊은 학습은 기계 학습의 하위 세트입니다.",
    "GM 판매자 인 로베로는 매일 글을 쓰는 오넬라 무티와 미친 사랑에 빠져 있습니다."
]

print("EN→KO chrF Score:")
chrf_ko = sacrebleu.corpus_chrf(hypotheses_ko, [references_ko])
print(chrf_ko)


# 한국어 → 영어 평가
references_en = [
    "The weather is very good today.",
    "I am studying machine learning.",
    "Artificial intelligence is changing the world.",
    "Deep learning is a subset of machine learning.",
    "Roberto, a GM salesman, is madly in love with Ornella Muti, to whom he writes every day."
]

hypotheses_en = [
    "The weather is very good today.",
    "I am studying machine learning.",
    "Artificial intelligence changes the world.",
    "Deep learning is a subset of machine learning.",
    "Roberto, a Jim-seller, loves Ornella Muti, whose letters he writes every day."
]

print("\nKO→EN chrF Score:")
chrf_en = sacrebleu.corpus_chrf(hypotheses_en, [references_en])
print(chrf_en)

EN→KO chrF Score:
chrF2 = 31.88

KO→EN chrF Score:
chrF2 = 77.53
